In [1]:
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio import Entrez
import tempfile
import os

C:\Users\ralme\anaconda3\envs\BIO\Lib\site-packages\Bio\Application\__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(


# Búsqueda BLAST online y local con Biopython

En este ejercicio se desarrolla un programa en Python que realiza una búsqueda de similitud entre una secuencia de ADN introducida por el usuario y una base de datos de nucleótidos. Para ello se utiliza BLASTn, una herramienta diseñada específicamente para comparar secuencias de ADN y encontrar regiones de homología con otras secuencias almacenadas en bases de datos públicas o locales.

El objetivo es mostrar por pantalla tres elementos clave:
1. El número total de resultados devueltos por BLAST.
2. El E-value del mejor alineamiento, que mide la significación estadística del resultado
3. La descripción de la secuencia más similar encontrada en la base de datos.

Estos resultados permiten identificar rápidamente si la secuencia introducida corresponde a un gen conocido, si presenta similitud parcial con secuencias de organismos relacionados o si no tiene coincidencias significativas.

## 1. Validación de la secuencia introducida por el usuario

El programa comienza solicitando una secuencia de ADN por teclado. Antes de enviarla a BLAST, se realiza un proceso de validación indispensable para evitar errores:

- Se eliminan espacios, saltos de línea y caracteres invisibles.
- Se convierten todos los caracteres a mayúsculas.
- Se verifica que la secuencia contenga únicamente A, C, G o T.

Si la secuencia contiene símbolos no válidos, el programa muestra un mensaje de error y detiene la ejecución.
Este paso asegura que la consulta enviada al servidor BLAST sea correcta y que no se produzcan fallos durante el análisis.



In [2]:
Entrez.email = "raquel.almeida105@alu.ulpgc.es"

def limpiar_secuencia(seq: str) -> str:
    limpia = seq.replace(" ", "").replace("\n", "").replace("\r", "").upper()
    validos = set("ACGT")
    if not limpia:
        raise ValueError("La secuencia está vacía.")
    if not set(limpia).issubset(validos):
        raise ValueError(
            "La secuencia contiene caracteres no válidos. Solo se permiten A, C, G, T, N."
        )
    return limpia


def resumen_resultados_blast(record):
    num_hits = len(record.alignments)
    if num_hits == 0:
        return 0, None, None

    mejor_aln = record.alignments[0]
    mejor_hsp = mejor_aln.hsps[0]

    mejor_evalue = mejor_hsp.expect
    descripcion = mejor_aln.hit_def

    return num_hits, mejor_evalue, descripcion

## 2. Búsqueda BLASTn online con Biopython

En esta parte se hace una consulta BLAST **online** contra los servidores de NCBI.

Para realizar la consulta online, se utiliza la función `NCBIWWW.qblast` indicando al servidor del NCBI:
- Programa: `blastn` (para ADN).
- Base de datos: `nt` (nucleótidos).

Luego, lee el restulado en formato XML con `NCBIXML.read` y utiliza la función `resumen_resultados_blast` para obtener el número total de hits, el mejor E-value, y la descripción del mejor hit. Finalmente, muestra esta información por pantalla.

In [3]:
seq_usuario = input("Introduce una secuencia de ADN: ")

try:
    seq_limpia = limpiar_secuencia(seq_usuario)
except ValueError as e:
    print(f"Error en la secuencia: {e}")
else:
    print("\nRealizando búsqueda BLAST online (esto puede tardar un poco)...")
    
    result_handle = NCBIWWW.qblast(
        program="blastn",
        database="nt",
        sequence=seq_limpia
    )
    
    blast_record = NCBIXML.read(result_handle)
    
    num_hits, mejor_evalue, descripcion = resumen_resultados_blast(blast_record)

    print("\n--- RESULTADOS BLAST ONLINE ---")
    print(f"Número de resultados (hits): {num_hits}")
    if num_hits > 0:
        print(f"Mejor E-value: {mejor_evalue}")
        print(f"Descripción del mejor hit: {descripcion}")
    else:
        print("No se han obtenido resultados significativos.")


Realizando búsqueda BLAST online (esto puede tardar un poco)...

--- RESULTADOS BLAST ONLINE ---
Número de resultados (hits): 0
No se han obtenido resultados significativos.


## 2. Preparación y uso de una base de datos BLAST local

Ahora se va a hacer lo mismo, pero usando BLAST de forma **local**.  
Para esta parte es necesario:

- Tener instaladas las herramientas **BLAST+**, como `blastn`, `makeblastdb`, etc..
- Tener creada una **base de datos local** con `makeblastdb -in secuencias.fasta -dbtype nucl -out mi_db_nucl`.

Este comando genera los archivos índice necesarios (.nin, .nsq, .nhr) que permiten que BLAST realice la búsqueda sin conexión a Internet.

Una vez creada la base de datos, el programa utiliza la clase `NcbiblastnCommandline` para ejecutar el comando blastn desde Python y almacenar los resultados en un archivo XML temporal.

La estructura del análisis local es idéntica a la del análisis online:,se parsea el resultado, se cuentan los hits y se identifican el mejor E-value y la mejor descripción.

Para ambos escenarios (online y local) se emplea la función `resumen_resultados_blast`, que recibe el informe BLAST parseado y devuelve:
- el número total de resultados obtenidos
- el E-value del mejor alineamiento
- la descripción de la secuencia más parecida

Si el número de hits es 0, la función devuelve valores nulos y el programa informa de que no existen coincidencias significativas.


In [5]:
BLASTN_EXE = "blastn"        
DB_LOCAL = "mi_db_nucl"

try:
    seq_limpia
except NameError:
    print("\nNo se puede ejecutar BLAST local porque la secuencia no es válida.")
else:
    print("\nRealizando búsqueda BLAST local...")
    
    with tempfile.TemporaryDirectory() as tmpdir:
        fasta_path = os.path.join(tmpdir, "query.fasta")
        xml_out = os.path.join(tmpdir, "blast_local.xml")

        with open(fasta_path, "w") as f:
            f.write(">consulta_usuario\n")
            f.write(seq_limpia + "\n")

        blast_cline = NcbiblastnCommandline(
            cmd=BLASTN_EXE,
            query=fasta_path,
            db=DB_LOCAL,
            evalue=0.001,
            outfmt=5,          
            out=xml_out
        )

        stdout, stderr = blast_cline()

        if not os.path.exists(xml_out):
            print("No se ha generado el archivo de salida de BLAST local. Revisa la configuración.")
        else:
            with open(xml_out) as result_handle_local:
                blast_record_local = NCBIXML.read(result_handle_local)

            num_hits_loc, mejor_evalue_loc, descripcion_loc = resumen_resultados_blast(blast_record_local)

            print("\n--- RESULTADOS BLAST LOCAL ---")
            print(f"Número de resultados (hits): {num_hits_loc}")
            if num_hits_loc > 0:
                print(f"Mejor E-value: {mejor_evalue_loc}")
                print(f"Descripción del mejor hit: {descripcion_loc}")
            else:
                print("No se han obtenido resultados significativos en la búsqueda local.")


Realizando búsqueda BLAST local...

--- RESULTADOS BLAST LOCAL ---
Número de resultados (hits): 0
No se han obtenido resultados significativos en la búsqueda local.


## Conclusiones finales

En la parte **online**, se usa `NCBIWWW.qblast`, que envía la consulta a los servidores de NCBI.  
  Esto es cómodo pero depende de la conexión a Internet y del tiempo de respuesta de NCBI.

En la parte **local**, se ejecuta `blastn` sobre una base de datos que tenemos en nuestro ordenador.  
  Esto requiere más preparación (instalar BLAST+, crear la base de datos local), pero:
  - Evita saturar el servidor de NCBI.
  - Suele ser más rápido una vez que la base de datos está preparada.
  - Nos permite trabajar con bases de datos específicas (por ejemplo, solo genes de un organismo concreto).
 
En ambos casos, a partir de los resultados recogemos:

- El **número total de hits** (`len(record.alignments)`).
- El **mejor E-value**, que indica la significación estadística del alineamiento.
- La **descripción** del mejor hit, que nos ayuda a identificar rápidamente a qué secuencia se parece más nuestra consulta.

Con esto se muestra, por tanto, cómo integrar Python, Biopython y BLAST para automatizar búsquedas de similitud de secuencias tanto online como en local.